<a href="https://colab.research.google.com/github/deepshikha04yadav/Traffic-Gesture-Detection-for-Automotive/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics mediapipe opencv-python-headless torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 57.1 MB/s eta 0:00:00

In [ ]:
!pip install torch torchvision torchaudio opencv-python matplotlib tqdm

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled p

In [1]:
import pandas as pd
import os
import tensorflow as tf
import keras_cv

data_dir = "/content/Traffic-gestures"
csv_path = os.path.join(data_dir, "_annotations.csv")

df = pd.read_csv(csv_path)
df.head()


,filename,width,height,class,xmin,ymin,xmax,ymax
0,BB2-GFR2-WG_jpg.rf.b1f59c42074c4f91c257fabdd50...,640,640,gloves,164,395,195,442
1,BB2-GFR2-WG_jpg.rf.b1f59c42074c4f91c257fabdd50...,640,640,gloves,443,265,514,291
2,BB2-GFR2-WG_jpg.rf.b1f59c42074c4f91c257fabdd50...,640,640,vest,186,265,322,400
3,BB2-GFR2-WG_jpg.rf.b1f59c42074c4f91c257fabdd50...,640,640,uniform,162,267,447,613
4,BB2-SS2-WG_jpg.rf.b6fb9cbf2a9807a5193cd1dbfba0...,640,640,gloves,47,222,98,273


In [2]:
from collections import defaultdict

grouped = defaultdict(lambda: {"boxes": [], "classes": []})

label_map = {name:i for i,name in enumerate(df['class'].unique())}
class_names = list(label_map.keys())

for _, row in df.iterrows():
    filename = row['filename']
    grouped[filename]["boxes"].append(
        [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
    )
    grouped[filename]["classes"].append(
        label_map[row['class']]
    )

len(grouped)

348

In [3]:
def data_generator(img_dir, grouped_dict, img_size=(640,640)):
    for filename, ann in grouped_dict.items():
        img_path = os.path.join(img_dir, filename)
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = img / 255.0

        boxes = tf.ragged.constant(ann["boxes"], dtype=tf.float32)
        labels = tf.ragged.constant(ann["classes"], dtype=tf.int32)

        yield img, {"boxes": boxes, "classes": labels}


In [4]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(data_dir, grouped),
    output_signature=(
        tf.TensorSpec(shape=(640, 640, 3), dtype=tf.float32),
        {
            "boxes": tf.RaggedTensorSpec(shape=(None,4), dtype=tf.float32),
            "classes": tf.RaggedTensorSpec(shape=(None,), dtype=tf.int32)
        }
    )
)


In [5]:

backbone = keras_cv.models.ResNet50Backbone.from_preset("resnet50_imagenet")

model = keras_cv.models.RetinaNet(
    num_classes=len(class_names),
    bounding_box_format="xyxy",
    backbone=backbone
)

model.compile(
    classification_loss=keras_cv.losses.FocalLoss(from_logits=True),
    box_loss=keras_cv.losses.SmoothL1Loss(),
    optimizer=tf.keras.optimizers.Adam(1e-4)
)
train_dataset = train_dataset.shuffle(64).batch(2, drop_remainder=True).prefetch(tf.data.AUTOTUNE)


In [ ]:
model.fit(train_dataset, epochs=20)
model.save("/content/gesture_detector")


In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

detector = keras_cv.models.RetinaNet.from_preset(
    "resnet50_imagenet",
    num_classes=len(class_names),
    bounding_box_format="xyxy"
)
detector.load_weights("/content/gesture_detector")

def predict_image(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = tf.image.resize(img_rgb, (640,640)) / 255.0
    img_in = tf.expand_dims(img_resized, axis=0)

    preds = detector.predict({"images": img_in})

    boxes = preds["boxes"][0].numpy()
    classes = preds["classes"][0].numpy()
    scores = preds["confidence"][0].numpy()

    for box, cls, score in zip(boxes, classes, scores):
        if score < 0.5:
            continue

        x1, y1, x2, y2 = map(int, box)
        gesture = class_names[int(cls)]
        cv2.rectangle(img_rgb, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img_rgb, f"{gesture} {score:.2f}",
                    (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (255,0,0), 2)

    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()


ValueError: File format not supported: filepath=/content/gesture_detector. Keras 3 only supports V3 `.keras` and `.weights.h5` files, or legacy V1/V2 `.h5` files.